# Applying filters to an image with OpenCV

A filter can be defined as a function that takes an image as input and returns a modified version of the image as output (Szeliski, 2010). It can apply effects to the image,
extract useful information such as edges and corners, and implement techniques such
as smoothing and inpainting.

This notebook shows different filters which can easily be created and applied to an image.<br>
Following filers will be shown:
* 2D convolution
* Image Blurring (smoothing)
  - averaging
  - gaussian filtering
  - median filtering

The code below is based on the OpenCV-python tutorial [``Smoothing Images``](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html)

## Load Image

In [ ]:
# Load pictures from github repository to colab space

!curl --remote-name -H 'Accept: application/vnd.github.v3.raw' --location https://raw.githubusercontent.com/iubh/DLMAIEAIS01_DEV/master/data/cv_pictures/baboon.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  175k  100  175k    0     0   379k      0 --:--:-- --:--:-- --:--:--  378k


## Adding noise to image

With the function ``add_noise()`` it is possible to add either a ``salt-and-pepper`` effect or ``gaussian-noise`` to an image.

In [ ]:
# Function add_noise(image, mode, par_01, par_02)
#
# @ param image: original image to add noise to (grayscale)
# @ param mode:  noise adding mode -> 'gauss' / 'sp' (salt-and-pepper)
# @ param par_01: guass: 'mean' (default: 0)   | 'sp': ratio of white and black pixels (default: 0.2)
# @ param par_02: gauss: 'sigma (default: 100) | 'sp': amount of pixels to count in for randomly choosing (default: 0.5)

def add_noise(image, mode, par_01=None, par_02=None):
  if mode == "gauss":
    # Temporary array for random distribution
    gauss = np.zeros(image.shape, np.uint8) 
    
    mean = 0
    if par_01 != None:
      mean = par_01
    sigma = 100
    if par_02 != None:
      sigma = par_02

    # Create the random distribution    
    cv2.randn(gauss, mean, sigma) 
    # Add the noise to the original image
    noisy = cv2.add(image, gauss) 
    return noisy
    
  #---------------------------------------
  if mode == "sp":
    row,col = image.shape
    s_vs_p = 0.2
    
    if par_01 != None:
      s_vs_p = par_01
    amount = 0.5
    if par_02 !=  None:
      amount = par_02
    
    out = np.copy(image)
    # Color random pixels in white
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    out[coords] = 1
    
    # Color random pixels in black
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    out[coords] = 0
    return out

# load resources

In [ ]:
# Load modules
import cv2
import numpy as np

# A replacement for the cv2.imshow() function to use in jupyter-notebooks
from google.colab.patches import cv2_imshow 

# Pyplot for visualising comparisons
import matplotlib.pyplot as plt

# 2D Convolution

OpenCV provides a function, cv2.filter2D(), to convolve a kernel with an image. As an example, we will try an averaging filter on an image. A 5x5 averaging filter kernel can be defined as follows:

$$K =  \frac{1}{25} \begin{bmatrix} 1 & 1 & 1 & 1 & 1  \\ 1 & 1 & 1 & 1 & 1 \\ 1 & 1 & 1 & 1 & 1 \\ 1 & 1 & 1 & 1 & 1 \\ 1 & 1 & 1 & 1 & 1 \end{bmatrix}$$

Filtering with the above kernel results in the following being performed: for each pixel, a 5x5 window is centered on this pixel, all pixels falling within this window are summed up, and the result is then divided by 25. This equates to computing the average of the pixel values inside that window. This operation is performed for all the pixels in the image to produce the output filtered image.

**[source: //opencv-python-tutroals.readthedocs.io](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html)**

In [ ]:
img = cv2.imread('baboon.jpg')

# Create Kernel matrix
kernel = np.ones((5,5),np.float32)/25

# Filter image and save result in another image
dst = cv2.filter2D(img,-1,kernel)

# Show 2 images in one row
plt.subplots(1,2, figsize =(15,15),sharey=True)

# Original picture
plt.subplot(121)
im1_conv = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(im1_conv)
plt.title('Original image'), plt.xticks([]), plt.yticks([])

# Filtered picture
plt.subplot(122)
im2_conv = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
plt.imshow(im2_conv)
plt.title('2D-Filtered image'), plt.xticks([]), plt.yticks([])

plt.show()

<Figure size 1080x1080 with 2 Axes>

# Image Blurring (Smoothing)

Image blurring is achieved by convolving the image with a low-pass filter kernel.<br> It is useful for removing noise. It actually removes high frequency content (e.g: noise, edges) from the image resulting in edges being blurred when this is filter is applied. (Well, there are blurring techniques which do not blur edges). OpenCV provides mainly four types of blurring techniques.

**[source: //opencv-python-tutroals.readthedocs.io](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html)**

### Averaging
This is done by convolving the image with a normalized box filter. It simply takes the average of all the pixels under kernel area and replaces the central element with this average. This is done by the function cv2.blur() or cv2.boxFilter(). Check the docs for more details about the kernel. We should specify the width and height of kernel. A 3x3 normalized box filter would look like this:

$$K =  \frac{1}{9} \begin{bmatrix} 1 & 1 & 1  \\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix}$$

**[source: //opencv-python-tutroals.readthedocs.io](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html)**

In [ ]:
# Load original image
img = cv2.imread('baboon.jpg')

# Load filter kernel
blur = cv2.blur(img,(5,5))

# Show 2 images in one row
plt.subplots(1,2, figsize =(15,15),sharey=True)

# Original picture
plt.subplot(121)
im1_conv = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(im1_conv)
plt.title('Original image'), plt.xticks([]), plt.yticks([])

# Filtered picture
plt.subplot(122)
im2_conv = cv2.cvtColor(blur, cv2.COLOR_BGR2RGB)
plt.imshow(im2_conv)
plt.title('Blurred image'), plt.xticks([]), plt.yticks([])

plt.show()

<Figure size 1080x1080 with 2 Axes>

### Gaussian Filtering
In this approach, instead of a box filter consisting of equal filter coefficients, a Gaussian kernel is used. It is done with the function, ``cv2.GaussianBlur()``. We should specify the width and height of the kernel which should be positive and odd. We also should specify the standard deviation in the X and Y directions, sigmaX and sigmaY respectively. If only sigmaX is specified, sigmaY is taken as equal to sigmaX. If both are given as zeros, they are calculated from the kernel size. Gaussian filtering is highly effective in removing Gaussian noise from the image.

**[source: //opencv-python-tutroals.readthedocs.io](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html)**

In [ ]:
# Load original image
img = cv2.imread('baboon.jpg')

# Load Gaussian-Blurr kernel
blur_gs = cv2.GaussianBlur(img, (5,5), 0)

# Show 2 images in one row
plt.subplots(1,2, figsize =(15,15),sharey=True)

# Original picture
plt.subplot(121)
im1_conv = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(im1_conv)
plt.title('Original image'), plt.xticks([]), plt.yticks([])

# Filtered picture
plt.subplot(122)
im2_conv = cv2.cvtColor(blur_gs, cv2.COLOR_BGR2RGB)
plt.imshow(im2_conv)
plt.title('Gaussian-Blurred image'), plt.xticks([]), plt.yticks([])

plt.show()

<Figure size 1080x1080 with 2 Axes>

### Median Filtering
Here, the function ``cv2.medianBlur()`` computes the median of all the pixels under the kernel window and the central pixel is replaced with this median value. This is highly effective in removing salt-and-pepper noise. One interesting thing to note is that, in the Gaussian and box filters, the filtered value for the central element can be a value which may not exist in the original image. However this is not the case in median filtering, since the central element is always replaced by some pixel value in the image. This reduces the noise effectively. The kernel size must be a positive odd integer.

**[source: //opencv-python-tutroals.readthedocs.io](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html)**

In [ ]:
# Load original image
img = cv2.imread('baboon.jpg')

# Create filtered image. The ksize parameter must be odd and greater than 1.
median = cv2.medianBlur(img, 5)

# Show 2 images in one row
plt.subplots(1,2, figsize =(15,15),sharey=True)

# Original picture
plt.subplot(121)
im1_conv = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(im1_conv)
plt.title('Original image'), plt.xticks([]), plt.yticks([])

# Filtered picture
plt.subplot(122)  
im2_conv = cv2.cvtColor(median, cv2.COLOR_BGR2RGB)
plt.imshow(im2_conv)
plt.title('Median-Blurred image'), plt.xticks([]), plt.yticks([])

plt.show()

<Figure size 1080x1080 with 2 Axes>

### Salt-and-Pepper noise

With the function 'add_noise()' it is possible to create a salt-and-pepper effect on an image.<br>
Salt-and-pepper noise can be aplied only to greyscale images.

In [ ]:
# Reading the color image and convert it to greyscale image 
img = cv2.imread('baboon.jpg', cv2.IMREAD_GRAYSCALE) 

# add_noise(image, mode, par_01, par_02)
# @ param image: original image to add noise to (grayscale)
# @ param mode:  noise adding mode -> 'gauss' / 'sp' (salt-and-pepper)
# @ param par_01: guass: 'mean' (default: 0)   | 'sp': ratio of white and black pixels (default: 0.2)
# @ param par_02: gauss: 'sigma (default: 100) | 'sp': amount of pixels to count in for randomly choosing (default: 0.5)

img_sp = add_noise(img, 'sp')

# medianBlur() filters noise from the image. the ksize parameter must be odd and greater than 1.
median_sp = cv2.medianBlur(img_sp, 5)

# Show 2 images in one row
plt.subplots(1,2, figsize =(15,15),sharey=True)

# Original picture
plt.subplot(121)
im1_conv = cv2.cvtColor(img_sp, cv2.COLOR_BGR2RGB)
plt.imshow(im1_conv)
plt.title('Salt-And-Pepper image'), plt.xticks([]), plt.yticks([])

# Filtered picture
plt.subplot(122)
im2_conv = cv2.cvtColor(median_sp, cv2.COLOR_BGR2RGB)
plt.imshow(im2_conv)
plt.title('Median-Blurred image'), plt.xticks([]), plt.yticks([])

plt.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


<Figure size 1080x1080 with 2 Axes>

### Filtering gaussian noise

In [ ]:
# the noise adding can only be done with grayscaled images
img = cv2.imread('baboon.jpg', cv2.IMREAD_GRAYSCALE) 
#
# add_noise(image, mode, par_01, par_02)
# @ param image: original image to add noise to (grayscale)
# @ param mode:  noise adding mode -> 'gauss' / 'sp' (salt-and-pepper)
# @ param par_01: guass: 'mean' (default: 0)   | 'sp': ratio of white and black pixels (default: 0.2)
# @ param par_02: gauss: 'sigma (default: 100) | 'sp': amount of pixels to count in for randomly choosing (default: 0.5)
#
img_gs = add_noise(img, 'gauss', 0, 50) # add noise to image

# medianBlur() filters noise from the image. the ksize parameter must be odd and greater than 1.
median_gs = cv2.medianBlur(img_gs,  3)

# Show 2 images in one row
plt.subplots(1,2, figsize =(15,15),sharey=True)

# Original picture
plt.subplot(121)
im1_conv = cv2.cvtColor(img_gs, cv2.COLOR_BGR2RGB)
plt.imshow(im1_conv)
plt.title('Gaussian-noise image'), plt.xticks([]), plt.yticks([])

# Filtered picture
plt.subplot(122)
im2_conv = cv2.cvtColor(median_gs, cv2.COLOR_BGR2RGB)
plt.imshow(im2_conv)
plt.title('Median-Blurred image'), plt.xticks([]), plt.yticks([])

plt.show()

<Figure size 1080x1080 with 2 Axes>

Copyright © 2021 IU International University of Applied Sciences